## 1. Magenta github clone & Setup

In [ ]:
!git clone https://github.com/tensorflow/magenta.git
!pip install pretty_midi
cd magenta
!pip install -e .

In [75]:
import os
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
os.chdir("/content/magenta/magenta/models/music_vae/")

## 2. Training

In [ ]:
!python music_vae_train.py \
 --config=groovae_4bar \
 --run_dir=/tmp/groovae_4bar \
 --mode=train \
 --tfds_name=groove/4bar-midionly

## 3. Generating

In [ ]:
!python music_vae_generate.py \
 --config=groovae_4bar \
 --checkpoint_file=/tmp/groovae_4bar/train/model.ckpt-949 \
 --mode=sample \
 --num_outputs=5 \
 --output_dir=/tmp/generated

## 4. Testing

In [56]:
import numpy as np
import pretty_midi
from glob import glob
import tensorflow as tf
from IPython import display

In [60]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [69]:
i = 4
example_path = glob("/tmp/generated/*")
for path in example_path:
  print(path)
pm = pretty_midi.PrettyMIDI(example_path[i])

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

display_audio(pm)

/tmp/generated/groovae_4bar_sample_2022-05-02_070342-004-of-005.mid
/tmp/generated/groovae_4bar_sample_2022-05-02_070342-000-of-005.mid
/tmp/generated/groovae_4bar_sample_2022-05-02_070342-002-of-005.mid
/tmp/generated/groovae_4bar_sample_2022-05-02_070342-001-of-005.mid
/tmp/generated/groovae_4bar_sample_2022-05-02_070342-003-of-005.mid
